# Introduction
In the previous notebook we predicted the next character of a name only by looking at the previous character. Here we want to do something a bit more sophistcated, we want to use more of the context than a single character, and we will be doing it in different ways: bag-of-words and using a multilayer perceptron (MLP). The MLP approach will be based on the a [paper from 2003](https://www.youtube.com/redirect?event=video_description&redir_token=QUFFLUhqbGJ0RndnaG1JbUhHVURROWVQVFNuOWZwU01oQXxBQ3Jtc0ttbW5SQnhQN0ZMWWtHem5FVFA5dFFCdk02R29raDVBNlMxaXpxU3E1S2dReHAxcVRYQjN3bjZsM2ZLcjdkRG5oWTBnSU1OUjZsaThxSnZLdVpKWEFWTDgzZnd3QlNmQXRldFFxRjZ1ekdfUFV0ZnVTcw&q=https%3A%2F%2Fwww.jmlr.org%2Fpapers%2Fvolume3%2Fbengio03a%2Fbengio03a.pdf&v=TCH_1BHY58I), in which they predict the next word using the previous words.

We will be doing the following:

* TBD

# Libraries

In [2]:
%matplotlib inline
%config IPCompleter.use_jedi=False

In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Data
For creating the language models we use a dataset of the most common names from [ssa.gov](https://www.ssa.gov/oact/babynames/).

In [4]:
# Reading names into a list
with open('../../data/names.txt', 'r') as f:
    names = f.readlines()
    names = [name.strip() for name in names]

In [13]:
# Building the vocabulary (character to/from index)
chars = sorted(list(set(''.join(names))))
chr_to_idx = {s:i+1 for i,s in enumerate(chars)}; print(chr_to_idx)
chr_to_idx['.'] = 0
idx_to_chr = {i:s for s,i in chr_to_idx.items()}; print(idx_to_chr)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [ ]:
http://www.youtube.com/watch?v=TCH_1BHY58I&t=9m18s